# Views

Views are, as the name implies, a view on top of our data.  They do not store any data themselves, but simply provide the users with a centralised, reusable, and controlled manner of accessing our data.

These are a very good way of simplifying the data for less experienced users, allowing them to have a consistent view of the data without needing to know any complex functions or SQL.

They can also be used as a security measure allowing people to see only what we wish them to by hiding the raw tables underneath and exposing only the views.

#### Using a View

This can be best shown by the following example:

We have several unskilled developers in our reporting team.  They can write basic SELECT and WHERE clauses but cannot cope with JOINs or data manipulation such as CONCAT.

These developers are regularly asked to produce reports including both the employee's full name and their department.

Our raw tables look as follows:

In [ ]:
select *
from JupyterDatabase.hr.employee;

select *
from JupyterDatabase.hr.jobTitle;

select *
from JupyterDatabase.hr.department;

As we know what we are doing, we can easily ascertain a more 'report friendly' dataset as follows:

In [ ]:
select e.employeeID, e.title, e.firstName, e.lastName,
		concat_ws(' ', e.title, e.firstName, e.lastName) as fullname,
		t.jobTitle, d.department
from JupyterDatabase.hr.employee as e
join JupyterDatabase.hr.jobTitle as t
on e.jobTitleID = t.jobTitleID
join JupyterDatabase.hr.department as d
on t.departmentID = d.departmentID;

This is where a VIEW comes to the fore as we can wrap this logic up in a VIEW and our developers can use it as if it is a table.

Consider the following:

In [ ]:
select *
from JupyterDatabase.hr.vwEmployeeInfo;

This is exactly what we needed, but how did we create it?

The key is, once again, in the declarative nature of the SQL language as we 'create a view' and place the query definition inside:

<code>create view <i>viewName</i><br/>
as<br/>
&nbsp&nbsp&nbsp&nbsp<i>Query definition goes here</i><br/>
go</code>

Annoyingly for us, SQL Server does not allow the creation of temporary views using the same # naming methodology of a temporary table and therefore although we can access the view safely, we cannot drop and re-create it each time we run a demo due to possible concurrency issues.

Therefore I will provide the syntax for copy and paste purposes, but attempting to run the below will return a permissions error:

In [ ]:
drop view if exists hr.vwEmployeeInfo
go

create view hr.vwEmployeeInfo
as
	select e.employeeID, e.title, e.firstName, e.lastName,
			concat_ws(' ', e.title, e.firstName, e.lastName) as fullname,
			t.jobTitle, d.department
	from JupyterDatabase.hr.employee as e
	join JupyterDatabase.hr.jobTitle as t
	on e.jobTitleID = t.jobTitleID
	join JupyterDatabase.hr.department as d
	on t.departmentID = d.departmentID;
go

Note - There are two critical areas in the above query. Firstly, 'CREATE VIEW' must be the first statement in a batch and therefore must be preceded with 'GO'. Secondly, our 'does this already exist, if so, drop' statement has been altered from 'table' to 'view' accordingly.

Although we cannot run the above, we can easily see how these are created and the power they have to provide centralised and reusable code.

Imagine we now need to change our fullName function in order to change our fullname format from 'Mr John Smith' to showing 'Smith, John'.  We can do this by 'ALTERing our VIEW':

In [ ]:
alter view hr.vwEmployeeInfo
as
	select e.employeeID, e.title, e.firstName, e.lastName,
			concat_ws(' ', e.lastName + ',', e.firstName) as fullname,
			t.jobTitle, d.department
	from JupyterDatabase.hr.employee as e
	join JupyterDatabase.hr.jobTitle as t
	on e.jobTitleID = t.jobTitleID
	join JupyterDatabase.hr.department as d
	on t.departmentID = d.departmentID;
go

If we were to run this (which we will not within our demo environment but feel free to do so in your own), we would see that the definition immediately changes and therefore all of our developers' code automatically begins to use the new definition with minimal fuss or risk of error.

#### Views and Security

The additional advantage we have with the VIEW is that we can hide underlying tables and sensitive columns from developers and other users with ease.

For example, it is possible to deny access to a table yet to grant access to a view which looks at that same table.  Therefore we ensure that they cannot see a table with sensitive data but can still view some of the safe columns within.

In our example we would deny read access to the hr.employee table because it has Salary information.  But our view only shows employee names and therefore we could grant read access to the View.

#### View Limitations

Although these sound great (and they are very good for all the reasons listed above), they are not the golden bullet to all problems.

For example, we can only place a single statement within a VIEW which means that we cannot perform multiple operations or return more than one dataset.  Neither can we pass parameters to a view.

Several of these can be addressed by other objects we can utilise.